In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

City_ID           City Country                 Date    Lat     Lng  \
0        0         Hobart      AU  2020-07-09 16:12:21 -42.88  147.33   
1        1       Luderitz     NaN  2020-07-09 16:15:43 -26.65   15.16   
2        2  Moose Factory      CA  2020-07-09 16:15:43  51.26  -80.61   
3        3         Harper      LR  2020-07-09 16:15:43   4.38   -7.72   
4        4        Ushuaia      AR  2020-07-09 16:12:46 -54.80  -68.30   

   Max Temp  Humidity  Cloudiness  Wind Speed  
0     39.20        93           0        6.93  
1     62.60        67          96       13.87  
2     80.60        61           1        5.82  
3     78.04        81          96       11.97  
4     35.60        74          40        3.36

In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Center map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [12]:
preferred_cities_df.count()

City_ID       151
City          151
Country       150
Date          151
Lat           151
Lng           151
Max Temp      151
Humidity      151
Cloudiness    151
Wind Speed    151
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


City Country  Max Temp    Lat     Lng Hotel Name
2   Moose Factory      CA     80.60  51.26  -80.61           
3          Harper      LR     78.04   4.38   -7.72           
7           Kapaa      US     77.00  22.08 -159.32           
15       Lorengau      PG     82.35  -2.02  147.27           
25         Albany      US     84.99  42.60  -73.97           
28      Jamestown      US     82.40  42.10  -79.24           
30      Caravelas      BR     82.33 -17.71  -39.25           
35         Avarua      CK     75.20 -21.21 -159.78           
37        Cloquet      US     75.20  46.72  -92.46           
40         Bethel      US     84.99  41.37  -73.41

In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.head(10)

City Country  Max Temp    Lat     Lng  \
2   Moose Factory      CA     80.60  51.26  -80.61   
3          Harper      LR     78.04   4.38   -7.72   
7           Kapaa      US     77.00  22.08 -159.32   
15       Lorengau      PG     82.35  -2.02  147.27   
25         Albany      US     84.99  42.60  -73.97   
28      Jamestown      US     82.40  42.10  -79.24   
30      Caravelas      BR     82.33 -17.71  -39.25   
35         Avarua      CK     75.20 -21.21 -159.78   
37        Cloquet      US     75.20  46.72  -92.46   
40         Bethel      US     84.99  41.37  -73.41   

                                Hotel Name  
2                   Cree Village Eco Lodge  
3                              Screensaver  
7   Sheraton Kauai Resort at Coconut Beach  
15              Lorengau Harbourside Hotel  
25                                          
28          Hampton Inn & Suites Jamestown  
30                  Pousada dos Navegantes  
35                            Paradise Inn  
37              Super 8 by Wyndham Cloquet  
40           Courtyard by Marriott Danbury

In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} Â°F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))